In [51]:
# The code was removed by Watson Studio for sharing.

Project name is [{'name': 'Geospatial_Coordinates.csv', 'asset_id': 'ec985893-9010-4b9e-a9c7-c58d4bd0ac0d'}]


# The Battle of Neighborhood: Clustering Localities in Kochi City

## 1. INTRODUCTION

    Most of the people around the world loves travelling and travellers loves to explore new food varities when they visit new places. Providing highly rated restaurant with price range helps travellers to make good planning on their budget as well pick the venues based on it. By using machine learning techniques we are providing quick glance to the traveler about the venue and highly rated restaurant with affordable price range in that area. 
    Also this projects aims at business people who likes to open restaurant in a city based on the cluster of different category of restaurant and demand.
    
## 2. Data
    
     In this project we are analysing the data of Kochi city which is part of Kerala state. Our analysis is targeted at localities within 30km of Kochi city. We are making using of Foursquare APIs to get details of the venues in Kochi city and Zomato API is used for extracting the restaurant details, price range, user rating, category etc. 
    Using the name, latitude and longitude values obtained from the Foursquare API, we are using Zomato search API to fetch data from its database. The Zomato API allows to find only restaurants based on a search criteria using the name, latitude, longitude, etc.

    Various data cleaning methods are adopted here to get the final dataset.From Foursquare API , the following details of each locaities is retrieved:
* **Name:** The name of the venue 
* **Category:** The category type as defined by the Foursqaure API
* **Latitude:** The latitude value of the venue
* **Longitude:** The longitude value of the venue.

    From Zomato API, the following details of each restaurant is retrieved: 
* **Name:** The name of the restaurant
* **Locality:** The locality of the restaurant
* **Aggregate Rating:** The average rating of the restaurant given be users
* **Price range:** The price ranges of the restaurant 
* **Average cost for two:** The average cost for two people dining at the restaurant
* **Address**: Address of the resturants
* **Review count**: Total user rating count for a specific resturant
* **Latitude:** The latitude value of the restaurant
* **Longitude:** The longitude value of the restaurant
* **City:** City in which resturant belongs to 

## 3. Methodology
    
    Geolocator is used for finding the lattitude and longitude value of the Kochi city. Folium library is used for reterieving the map of Kochi City.

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: - 

In [ ]:
address = 'Kochi, Kerala'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of  kochi City are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of Kochi using latitude and longitude values
kochi_map = folium.Map(location=[latitude, longitude], zoom_start=12)

In [ ]:
kochi_map

Define Foursquare Credentials and Version

In [ ]:
CLIENT_ID = 'I32HE3AL43MTDKD4VBCKE0SZYVTA4EUR4KSHSVH30NT33KJO' # your Foursquare ID
CLIENT_SECRET = '21DSPVZQRUT5KG1MDQYUPAQ1JMGFHCJTJNITA4YD3C2EWUZD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Now, let's get the top venues  in Kochi City within a radius of 30 Kms

In [ ]:
def get_foursquare_api (offset):
    LIMIT = 100
    radius = 30000
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&offset={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        latitude, 
        longitude, 
        radius, 
        LIMIT,
        offset)
    results = requests.get(url).json()
    return results

In [ ]:
#Sample API call
results = get_foursquare_api (100)

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Foursquare API has a limit of fecthing 100 venues at a time and we are using offset for handling this

In [ ]:
kochi_venues = pd.DataFrame()
offset = 0
while True:
    results = get_foursquare_api (offset)
    venue_len = len(results['response']['groups'][0]['items'])
    print("Venue length is {}".format(venue_len))
    venues = results['response']['groups'][0]['items']
    
    nearby_venues = json_normalize(venues) # flatten JSON

    # filter columns
    filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
    nearby_venues =nearby_venues.loc[:, filtered_columns]

    # filter the category for each row
    nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

    # clean columns
    nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

    kochi_venues = kochi_venues.append(nearby_venues, ignore_index = True)
    if venue_len < 100:
        break
    else:
        offset = offset + 100
        

In [ ]:
print('{} venues were returned by Foursquare.'.format(kottayam_venues.shape[0]))

In [ ]:
# add markers to map
for lat, lng, label in zip(kochi_venues['lat'], kochi_venues['lng'], kochi_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(kochi_map)  
kochi_map

### Zomato API call
    Let us first define the user-key generated from the zomato developer account


In [ ]:
headers = {'user-key': '36a42554f93b2bd1a4dad82816dd9981'}

In [ ]:
# Sample API call
url = 'https://developers.zomato.com/api/v2.1/categories'
print(url)
zomato_results = requests.get(url, headers=headers).json()
zomato_results

Using the Zomato search API for fecthing the resturants corresponding to a venue and its lattitude and longitude values


In [ ]:
kochi_rest = pd.DataFrame()
for ind in kochi_venues.index: 
    url = ('https://developers.zomato.com/api/v2.1/search?q={}' + 
          '&start=0&count=20&lat={}&lon={}&sort=real_distance').format(kochi_venues['name'][ind], kochi_venues['lat'][ind], kochi_venues['lng'][ind])
    result = requests.get(url, headers = headers).json()
    print("Zomato details of venue {}".format(kochi_venues['name'][ind]))
    nearby_rest = json_normalize(result['restaurants'])
    kochi_rest = kochi_rest.append(nearby_rest, ignore_index = True)  

In [ ]:
#Let us save the datas to csv file as zomato REST API has daily REST call limit
kochi_rest.to_csv('/home/dsxuser/work/kochi_resturant.csv')

In [ ]:
# filter columns
filtered_columns = ['restaurant.name', 'restaurant.location.latitude', 'restaurant.location.longitude', 'restaurant.location.address', 'restaurant.location.city', 'restaurant.location.locality', 'restaurant.average_cost_for_two', 'restaurant.all_reviews_count', 'restaurant.price_range', 'restaurant.user_rating.aggregate_rating']
kochi_rest =kochi_rest.loc[:, filtered_columns]

# clean columns
kochi_rest.columns = [col.split(".")[-1] for col in kochi_rest.columns]


In [ ]:
kochi_rest['name'].value_counts()

## Data Cleaning

Removing the duplicate resturant name from the data frame

In [ ]:
# dropping duplicate values 
kochi_rest.drop_duplicates(keep=False,inplace=True)

In [ ]:
kochi_rest.dtypes

In [ ]:
# Converting the object format of lattitude, longitude and aggregate_rating to real numbers
convert_dict = {'latitude': float, 
                'longitude': float,
                'aggregate_rating': float
               } 
kochi_filtered_venues = kochi_rest.astype(convert_dict) 

Let us remove the rows which is having review count as zero and customer review rating as zero, inorder to make our analysis easier

In [ ]:
kochi_filtered_venues = kochi_filtered_venues[(kochi_filtered_venues['all_reviews_count'] > 0) & (kochi_filtered_venues['aggregate_rating'] > 0)].reset_index()


In [ ]:
kochi_filtered_venues = kochi_filtered_venues.drop(['index'], axis=1)


In [ ]:
kochi_filtered_venues

### Data Analysis

#### A. Number of Restaurants in each Locality

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors


resturant_distribution = kochi_filtered_venues['locality'].value_counts()
colors = cm.rainbow(np.linspace(0, 1, len(resturant_distribution.index)))
plt.figure(figsize = (20, 12))
plt.xticks(rotation = 90)
plt.xlabel("Locality", fontsize = 16)
plt.ylabel("Count", fontsize = 16)
plt.title("Count of Resturants in each locality", fontsize = 16)
plt.bar(resturant_distribution.index, resturant_distribution.values, color = colors)

#### B. Average price for two in restaurants in each locality:

In [ ]:
price_for_two = kochi_filtered_venues[['locality', 'average_cost_for_two']]
cost_disctribution = price_for_two.groupby(['locality']).mean()


In [ ]:
colors = cm.rainbow(np.linspace(0, 1, len(cost_disctribution.index)))
plt.figure(figsize = (20, 12))
plt.xticks(rotation = 90)
plt.xlabel("Locality", fontsize = 16)
plt.ylabel("Mean cost for two", fontsize = 16)
plt.title("Mean cost for two in each locality", fontsize = 16)
plt.bar(cost_disctribution.index, cost_disctribution['average_cost_for_two'], color = colors)

#### C.Mean of aggregate ratings of resturants in each localities

In [ ]:
agg_rating = kochi_filtered_venues[['locality', 'aggregate_rating']]
agg_distribution = agg_rating.groupby(['locality']).mean()

In [ ]:
colors = cm.rainbow(np.linspace(0, 1, len(agg_distribution.index)))
plt.figure(figsize = (20, 12))
plt.xticks(rotation = 90)
plt.xlabel("Locality", fontsize = 16)
plt.ylabel("Mean aggregate rating", fontsize = 16)
plt.title("Mean aggregate ratings in each locality", fontsize = 16)
plt.bar(agg_distribution.index, agg_distribution['aggregate_rating'], color = colors)

In [ ]:
rest_rating = kochi_filtered_venues['aggregate_rating'].value_counts().sort_index()
plt.figure(figsize = (20, 12))
plt.bar(rest_rating.index, rest_rating.values, color = cm.rainbow(np.linspace(0, 1, len(rest_rating.index))))
plt.xlabel("Aggregate Rating", fontsize = 16)
plt.ylabel("Count", fontsize = 16)
plt.title("Count of localities", fontsize = 16)

The above bar diagram shows that most of localities in Kochi has **ratings of 3.5**

In [ ]:
# create map of Kochi using latitude and longitude values
kochi_rest_map = folium.Map(location=[latitude, longitude], zoom_start=12)
bins = [1.0, 2.0, 3.0, 4.0, 5.0]
labels = ['Very Low', 'Low', 'Good', 'Very good']
kochi_filtered_venues['rating_bin'] = pd.cut(kochi_filtered_venues['aggregate_rating'], bins = bins, labels = labels, include_lowest = True)

In [ ]:
color_map = {'Very Low': 'red', 'Low': 'orange', 'Good': 'green', 'Very good': 'darkgreen'}

In [ ]:
# add markers to map
for locality, address, latitude, longitude, rating_bin in zip(kochi_filtered_venues['locality'], 
                                                          kochi_filtered_venues['address'], 
                                                          kochi_filtered_venues['latitude'], 
                                                          kochi_filtered_venues['longitude'], 
                                                          kochi_filtered_venues['rating_bin']):
    label = '{}, {}'.format(locality, address)
    label = folium.Popup(label, parse_html = True)
    folium.Marker(
        [latitude, longitude],
        icon = folium.Icon(color = color_map[rating_bin]),
        popup = label).add_to(kochi_rest_map)  

kochi_rest_map.save("/home/dsxuser/work/kochi_rest_map.html")
kochi_rest_map

## Clustering

K-means clustering algorithm is used for clustering the localities based on price for two and aggregate rating

In [ ]:

from sklearn.cluster import KMeans

NO_OF_CLUSTERS = 3

kochi_cluster = kochi_filtered_venues.drop(['name', 'address', 'city', 'locality', 'all_reviews_count', 'price_range', 'rating_bin'], 1)
kMeans = KMeans(n_clusters = NO_OF_CLUSTERS, random_state = 0).fit(kochi_cluster)
kochi_filtered_venues.insert(0, 'cluster_labels', kMeans.labels_)


In [ ]:
kochi_cluster_map = folium.Map(location = [latitude, longitude], zoom_start = 12)
color_map = { 0: 'green', 1: 'orange', 2: 'blue'}

# add venues to the map
markers_colors = []
for venue, address, cluster, latitude, longitude in zip(kochi_filtered_venues['locality'], 
                                                        kochi_filtered_venues['address'], 
                                                        kochi_filtered_venues['cluster_labels'], 
                                                        kochi_filtered_venues['latitude'], 
                                                        kochi_filtered_venues['longitude']):
    label = folium.Popup(str(venue) + ', ' + str(address), parse_html = True)
    folium.CircleMarker(
        [latitude, longitude],
        radius = 5,
        popup = label,
        color = color_map[cluster],
        fill = True,
        fill_color = color_map[cluster],
        fill_opacity = 0.7).add_to(kochi_cluster_map)

# add cluster centers to the map
for index, cluster in enumerate(kMeans.cluster_centers_):
    latitude = cluster[0]
    longitude = cluster[1]
    label = folium.Popup("Cluster: " + str(index), parse_html = True)
    folium.CircleMarker(
        [latitude, longitude],
        radius = 10,
        popup = label,
        color = color_map[index],
        fill = True,
        fill_color = color_map[index],
        fill_opacity = 0.7).add_to(kochi_cluster_map)

kochi_cluster_map.save("/home/dsxuser/work/kochi_cluster_map.html")
kochi_cluster_map

#### Cluster 1 details


In [ ]:
cluster_1 = kochi_filtered_venues[kochi_filtered_venues['cluster_labels'] == 0]
print("Cluster -1 Analysis")
cluster_1.reset_index(drop = True)

In [ ]:
print("Mean of cluster -1 for average_cost_for_two {} aggregate_rating {}".format(cluster_1['average_cost_for_two'].mean(), cluster_1['aggregate_rating'].mean()))


In [ ]:
cluster_2 = kochi_filtered_venues[kochi_filtered_venues['cluster_labels'] == 1]
print("Cluster -2 Analysis")
cluster_2.reset_index(drop = True)

In [ ]:
print("Mean of cluster -2 for average_cost_for_two {} aggregate_rating {}".format(cluster_2['average_cost_for_two'].mean(), cluster_2['aggregate_rating'].mean()))

In [ ]:
cluster_3 = kochi_filtered_venues[kochi_filtered_venues['cluster_labels'] == 2]
print("Cluster -3 Analysis")
cluster_3.reset_index(drop = True)
print("Mean of cluster -3 for average_cost_for_two {} aggregate_rating {}".format(cluster_3['average_cost_for_two'].mean(), cluster_3['aggregate_rating'].mean()))

## 4. RESULTS AND DISCUSSIONS

The data analysis performed here helps visitor to take good decsision based on there budget and preferences while visiting **Kochi, Kerala**. Below are the some of the conclusion which we can derived based on above analysis.
* Resturant counts are more in area close to **Edappally** and **Kaloor**
* The most costlier resturant in Kochi are in locality **Kochi Marriott Hotel Edappally**, **Fargrant FortKochi and** **Malabar Resorts & Spa, Wellingdon Island**
* **Kaloor** locality finds to be more budget friendly, has good eateries count and has good aggregate ratings as well so good place to visit

Below are the conclusion we can derive from the K-means clustering algorithm
* Localities coming under cluster-2 is having **avg cost for two = 306 and rating = 3.48**
* Localities coming under cluster-1 is having **avg cost for two = 807 and rating = 3.68**
* Localities coming under cluster-3 is having **avg cost for two = 1930 and rating = 3.71**

Most of the localities in centre of kochi city is falling under cluster-2. Most expensive localities in Kochi has good customer ratings.

## 5. CONCLUSION

In this project we are making use of Foursquare and Zomato APIs to explore insight into the eateries in Kochi city. 
The data analysis performed here can be used by the new visitors 
who likes to visit the Kochi and plan their budget based on the data available here. 
The data can be used by the Business people who has plan to start the bussiness as it provides 
the cluster distribution of eateries and its ratings as well as price for two people. The locality **Kaloor**
finds to be budget friendly and most of the eateries avaialble in Kochi has good customer rating. So its a reasonable
choice to visit Kochi city


In [ ]:
!pwd